In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
df = pd.read_csv("/content/drive/MyDrive/temperature_CO2/temperature_CO2_preprocessing.csv",
                 index_col=0)

In [26]:
df.head()

,year,Unixtime,world_temp,CO2/ppm,1_CO2,2_CO2
0,1986,504576000,-0.42,347.2,345.7,344.3
1,1987,536112000,-0.27,349.0,347.2,345.7
2,1988,567648000,-0.28,351.3,349.0,347.2
3,1989,599184000,-0.35,353.0,351.3,349.0
4,1990,630720000,-0.19,354.1,353.0,351.3


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        30 non-null     int64  
 1   Unixtime    30 non-null     int64  
 2   world_temp  30 non-null     float64
 3   CO2/ppm     30 non-null     float64
 4   1_CO2       30 non-null     float64
 5   2_CO2       30 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 1.6 KB


In [28]:
X = df.drop(["year", "world_temp"], axis=1)
y = df["world_temp"]

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.2,
                                                    shuffle=True,
                                                    random_state=0)

In [30]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import lightgbm as lgb

kf = KFold(n_splits=5, shuffle=True, random_state=1)

params = {
    "objective": "rmse",
    "metrics": "rmse",
    "learning_rate": .005,
    "verbose": -1,
    "random_state": 1,
}

models = []
valid_scores = []
oof = np.zeros(len(X_train))

In [31]:
for fold, (tr_idx, va_idx) in enumerate(kf.split(X=X_train, y=y_train)):
    X_tr = X_train.iloc[tr_idx, :]
    y_tr = y_train.iloc[tr_idx]
    X_va = X_train.iloc[va_idx, :]
    y_va = y_train.iloc[va_idx]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_va, y_va, reference=lgb_train)

    model = lgb.train(params=params,
                      train_set=lgb_train,
                      num_boost_round=10000,
                      valid_sets=[lgb_train, lgb_eval],
                      valid_names=["train", "vaild"],
                      callbacks=[lgb.early_stopping(100),
                                 lgb.log_evaluation(500)])

    y_va_pred = model.predict(X_va, num_iteration=model.best_iteration)
    score = np.sqrt(mean_squared_error(y_va, y_va_pred))
    print(f"fold:{fold+1}, RMSE valid:{score:.3f}")
    print("-------------------------------------------------------------------")

    valid_scores.append(score)
    models.append(model)
    oof[va_idx] = y_va_pred

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	train's rmse: 0.1781	vaild's rmse: 0.154332
fold:1, RMSE valid:0.154
-------------------------------------------------------------------
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	train's rmse: 0.165404	vaild's rmse: 0.21297
fold:2, RMSE valid:0.213
-------------------------------------------------------------------
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	train's rmse: 0.187714	vaild's rmse: 0.0998117
fold:3, RMSE valid:0.100
-------------------------------------------------------------------
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	train's rmse: 0.17533	vaild's rmse: 0.182241
fold:4, RMSE valid:0.182
-------------------------------------------------------------------
Training until validation scores don't impr

In [34]:
cv_score = np.mean(valid_scores)
print(f"CV score: {cv_score:.3f}")

CV score: 0.184


In [33]:
0.196**2

0.038416000000000006